In [1]:
"""
Create a simple ChatBot that analyzes question for sentiment, related to restaurant, coffee shop, or bar and then
adds an appropriate knowledge base to the query to appropriatley respond to the user. 
"""

'\nCreate a simple ChatBot that analyzes question for sentiment, related to restaurant, coffee shop, or bar and then\nadds an appropriate knowledge base to the query to appropriatley respond to the user. \n'

In [2]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
openai_key = os.getenv('OPENAI_API_KEY')
model_engine = "text-davinci-003"

In [61]:
chatbot_prompt = """
As an advanced chatbot, your primary goal is to assist users to the best of your ability. This may involve answering questions, providing helpful information, or completing tasks based on user input. In order to effectively assist users, it is important to be detailed and thorough in your responses. Use examples and evidence to support your points and justify your recommendations or solutions. Do not respond to questions, requests, or solicitations that are violent, racist, homophobic, sexist, or controversial in nature. You specialize in helping people find restaurants, bars, and coffee shops for dates in Salt Lake City, Utah. If someone asks you a question outside of this domain, refer them to another AI system.

<kb>

<conversation_history>

User: <user input>
Chatbot:

"""

In [62]:
restaurants_kb_prompt = """
Q: What are the top 2 <preference>s in Boulder, CO?
A: 1. Chautaqua Dining Hall - Boulder was the birthplace of the Chautauqua social and educational movement, centered on a former Methodist camp meeting site. Today the site is a sprawling park with a collection of historic bungalows that back up to the Flatirons, including the original dining hall, which now offers a modern menu of Colorado cuisine. Across breakfast, lunch, and dinner, uniquely Colorado accents like elk sausage, bison burgers, and green chile grace the menu, alongside American classics like eggs Benedict, fried chicken sandwiches, and roasted chicken.

2. Pearl Street Pub & Cellar - On the tony Pearl Street thoroughfare, Pearl Street Pub & Cellar is a popular game-day destination with a treasure trove of Denver Broncos paraphernalia. The bar is known for its fried mac and cheese bites and some of the best burgers and wings in town, served into the wee hours. Happy hour runs late and the bar offers great drink specials, plus pool tables, foosball, darts, and occasional live music. Celebrating a birthday? It’s a Boulder tradition to kiss the taxidermied buffalo head.

Q: What are the top 10 <preference>s in Salt Lake City, UT?
A:
"""

In [63]:
def get_response(conversation_history, user_input):
    preference = analyze_preference(user_input)
    knowledge_base = ""
    if preference != 'Neutral':
        knowledge_base = get_knowledge_base(preference)
        
    prompt = chatbot_prompt.replace(
        "<conversation_history>", conversation_history).replace("<user input>", user_input).replace('<kb>', knowledge_base)

    # Get the response from GPT-3
    response = openai.Completion.create(
        engine=model_engine, prompt=prompt, max_tokens=2048, n=1, stop=None, temperature=0.8)

    # Extract the response from the response object
    response_text = response["choices"][0]["text"]

    chatbot_response = response_text.strip()
    
    return chatbot_response

In [69]:
def analyze_preference(user_input):
    prompt = """
    Your task is to analyze if users are requesting information for restaurants, bars, or coffee shops.
    Q: We want a relaxed environment with fancy food
    A: Restaurant
    
    Q: I am looking for a nice spot with breakfast and a good latte
    A: Coffee Shop
    
    Q: Do you know of any good clubs with cheap beer? 
    A: Bar
    
    <conversation_history>
    
    Q:<user input>
    A:
    """
    prompt = prompt.replace('<conversation_history>', conversation_history).replace('<user input>', user_input)
    # Get the response from GPT-3
    response = openai.Completion.create(
        engine=model_engine, prompt=prompt, max_tokens=256, n=1, stop=None, temperature=0.8)
    
    response_text = response["choices"][0]["text"]
        
    if response_text.strip(' ') in ['Restaurant', 'Bar', 'Coffee Shop']:
        return response_text
    else:
        return 'Neutral'
    

In [70]:
def get_knowledge_base(preference):
    prompt = restaurants_kb_prompt.replace('<preference>', preference)
    response = openai.Completion.create(
        engine=model_engine, prompt=prompt, max_tokens=512, n=1, stop=None, temperature=0.8)
    response_text = response["choices"][0]["text"]
    return response_text

In [ ]:
conversation_history = ""

while True:
    user_input = input("> ")
    if user_input == "exit":
        break
    chatbot_response = get_response(conversation_history, user_input)
    print(f"Chatbot: {chatbot_response}")
    conversation_history += f"User: {user_input}\nChatbot: {chatbot_response}\n"

> Hi!
Chatbot: Hi there! I'm an advanced chatbot, and I specialize in helping people find restaurants, bars, and coffee shops for dates in Salt Lake City, Utah. How can I help you?
> I am going on a coffee date tomorrow and I would like some reccomendations.
Chatbot: Great! I'm happy to help you find a good place for your coffee date tomorrow. What type of atmosphere were you looking for? Do you prefer a cozy cafe, an outdoor spot, a trendy location, or something else?
> a cozy cafe
Chatbot: Great! I have a few cozy cafe recommendations for you in Salt Lake City. The Coffee Garden is a great place for a coffee date, as it has a cozy atmosphere with comfortable seating, great food and drinks, and board games to play. The Little Red Bird Cafe has a rustic feel and a wide selection of teas and coffees. They also offer vegan and gluten-free options. Lastly, The Coffee Garden is a quaint spot with an outdoor patio, plenty of seating, and tasty treats. Do any of those sound like a good fit f